In [1]:
import pandas as pd
import os

In [2]:
# reading in original CSV from Kaggle 
noc_data = pd.read_csv('noc_regions.csv')
noc_data.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [3]:
# rename column to country so we have a common column in case a merge in pandas is necessary to a separate 
#    csv from a different datasource that includes lat long  
noc_data.rename(columns = {'region': 'country'}, inplace=True)
noc_data.head()

,NOC,country,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [4]:
# dropping unnecessary columns
noc_data.drop(['notes'], axis=1, inplace=True)

In [5]:
noc_data = noc_data.dropna(subset=['country'])  # col_list is a list of column names to consider for nan values.


In [6]:
# making a single replacement so we get matches for USA athletes related since 'USA' is not abbreviated
#     on geojson
noc_data['country'].replace(
    to_replace=['USA'],
    value='United States',
    inplace=True
)

In [7]:
noc_data.count()

NOC        227
country    227
dtype: int64

In [8]:
noc_data = noc_data.set_index('NOC')

In [9]:
noc_data

,country
NOC,
AFG,Afghanistan
AHO,Curacao
ALB,Albania
ALG,Algeria
AND,Andorra
...,...
YEM,Yemen
YMD,Yemen
YUG,Serbia


In [10]:
noc_data.dtypes

country    object
dtype: object

In [11]:
noc_data.to_csv('noc_data_clean.csv',encoding='utf-8',index=True)

In [12]:
# reading in original CSV from Kaggle 
athlete_data = pd.read_csv('athlete_events.csv')
athlete_data.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [13]:
# dropping columns that aren't necessary leaving NaN's in Medal and Age section for now as we still want to see all athlete
#     regardless of metal/ages.
athlete_data.drop(['Height', 'Weight', 'Team','Games'], axis=1, inplace=True)
athlete_data.set_index('ID')

,Name,Sex,Age,NOC,Year,Season,City,Sport,Event,Medal
ID,,,,,,,,,,
1,A Dijiang,M,24.0,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
2,A Lamusi,M,23.0,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
3,Gunnar Nielsen Aaby,M,24.0,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
4,Edgar Lindenau Aabye,M,34.0,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
5,Christine Jacoba Aaftink,F,21.0,NED,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...
135569,Andrzej ya,M,29.0,POL,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
135570,Piotr ya,M,27.0,POL,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
135570,Piotr ya,M,27.0,POL,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN


In [14]:
# filling Medal NaN's w value
athlete_data['Medal'].fillna('None',inplace=True)

In [15]:
athlete_data['Name'].replace(
    to_replace=["'"],
    value="",
    inplace=True
)

In [16]:
athlete_data = athlete_data.drop_duplicates()

In [17]:
# athlete_data.dtypes
# athlete_data.nunique
# 
athlete_data.count()

ID        269718
Name      269718
Sex       269718
Age       260410
NOC       269718
Year      269718
Season    269718
City      269718
Sport     269718
Event     269718
Medal     269718
dtype: int64

In [21]:
merged_noc_athlete_data = pd.merge(noc_data, athlete_data, how='inner',on = 'NOC')

In [22]:
merged_noc_athlete_data

,NOC,country,ID,Name,Sex,Age,Year,Season,City,Sport,Event,Medal
0,AFG,Afghanistan,502,Ahmad Shah Abouwi,M,NaN,1956,Summer,Melbourne,Hockey,Hockey Men's Hockey,None
1,AFG,Afghanistan,1076,Jammal-ud-Din Affendi,M,28.0,1936,Summer,Berlin,Hockey,Hockey Men's Hockey,None
2,AFG,Afghanistan,1101,Mohammad Anwar Afzal,M,NaN,1948,Summer,London,Football,Football Men's Football,None
3,AFG,Afghanistan,1745,Mohammad Aktar,M,17.0,1980,Summer,Moskva,Wrestling,"Wrestling Men's Light-Flyweight, Freestyle",None
4,AFG,Afghanistan,4628,Mohammad Daoud Anwary,M,22.0,1964,Summer,Tokyo,Wrestling,"Wrestling Men's Bantamweight, Freestyle",None
...,...,...,...,...,...,...,...,...,...,...,...,...
269343,ZIM,Zimbabwe,130832,Hillary Wilson,F,15.0,1960,Summer,Roma,Swimming,Swimming Women's 100 metres Butterfly,None
269344,ZIM,Zimbabwe,130832,Hillary Wilson,F,15.0,1960,Summer,Roma,Swimming,Swimming Women's 4 x 100 metres Medley Relay,None
269345,ZIM,Zimbabwe,130880,Peter Arthur Wilson,M,20.0,1980,Summer,Moskva,Sailing,Sailing Mixed One Person Dinghy,None
269346,ZIM,Zimbabwe,131478,"Jennifer ""Jenny"" Wood",F,16.0,1964,Summer,Tokyo,Swimming,Swimming Women's 100 metres Butterfly,None


In [23]:
# create new CSV with just data of interest, this will be imported into DB
merged_noc_athlete_data.to_csv('merged_noc_athlete_data_clean.csv',encoding='utf-8',index=False)